# data engineering


---


## table of contents

1. loading packages
2. loading data
  1. renaming columns
  2. typing columns
    1. date time typing
3. data transformations
  1. converting foreign currency to usd
  2. misc. transformations
4. g


---





## 1.0 loading packages

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [2]:
import pandas as pd
import numpy as np
from plotnine import *

from sklearn.metrics import mean_squared_error, r2_score

# scientific notation setting to 2
pd.set_option('display.float_format', lambda x: '%.2f' % x)

## 2.0 loading data


In [3]:
app_activity = pd.read_csv("https://raw.githubusercontent.com/connoralydon/rosetta_stone_final/main/data/app_activity.csv")
subscriber_info = pd.read_csv("https://raw.githubusercontent.com/connoralydon/rosetta_stone_final/main/data/subscriber_info_raw.csv")
#churn = pd.read_csv("https://raw.githubusercontent.com/lizlyon/410FinalData/main/churn.csv") #need more info on this

#### 2.1 renaming columns names

 to _ separated and lowercase

In [4]:
app_activity.columns = app_activity.columns.str.replace(" ", "_")
app_activity.columns = app_activity.columns.str.lower()
app_activity.columns

Index(['id', 'app_session_platform', 'app_activity_type', 'app_session_date'], dtype='object')

In [5]:
subscriber_info.columns = subscriber_info.columns.str.replace(" ", "_")
subscriber_info.columns = subscriber_info.columns.str.lower()
subscriber_info = subscriber_info.rename(columns={'purchase_amount_(usd)':"purchase_amount_usd",
                                                  'purchase_amount':"purchase_amount_raw"}) #FIXME
subscriber_info.columns

Index(['id', 'language', 'subscription_type', 'subscription_event_type',
       'purchase_store', 'purchase_amount_raw', 'currency',
       'subscription_start_date', 'subscription_expiration', 'demo_user',
       'free_trial_user', 'free_trial_start_date', 'free_trial_expiration',
       'auto_renew', 'country', 'user_type', 'lead_platform',
       'email_subscriber', 'push_notifications', 'send_count', 'open_count',
       'click_count', 'unique_open_count', 'unique_click_count'],
      dtype='object')

In [6]:
app_activity.head()

,id,app_session_platform,app_activity_type,app_session_date
0,1,ios,App Launch,3/20/2019
1,2,android,App Launch,12/3/2019
2,3,ios,App Launch,5/2/2019
3,4,ios,App Launch,2/6/2020
4,5,ios,Start,9/14/2019


In [7]:
subscriber_info.head()

,id,language,subscription_type,subscription_event_type,purchase_store,purchase_amount_raw,currency,subscription_start_date,subscription_expiration,demo_user,...,country,user_type,lead_platform,email_subscriber,push_notifications,send_count,open_count,click_count,unique_open_count,unique_click_count
0,1,POR,Limited,INITIAL_PURCHASE,App,NaN,NaN,12/28/2018,6/28/2019,Yes,...,US/Canada,Consumer,App,Yes,Yes,63.00,7.00,0.00,6.00,0.00
1,2,EBR,Limited,INITIAL_PURCHASE,Web,39.00,USD,11/28/2019,2/28/2020,No,...,Other,Consumer,Web,No,Yes,4.00,3.00,0.00,1.00,0.00
2,3,ESP,Limited,INITIAL_PURCHASE,Web,0.00,USD,12/31/2018,12/31/2019,No,...,US/Canada,Consumer,Web,Yes,Yes,1.00,0.00,0.00,0.00,0.00
3,4,KOR,Limited,INITIAL_PURCHASE,App,NaN,NaN,11/7/2019,2/7/2020,Yes,...,US/Canada,Consumer,App,Yes,Yes,14.00,0.00,0.00,0.00,0.00
4,5,ENG,Limited,INITIAL_PURCHASE,App,NaN,NaN,8/13/2019,11/13/2019,No,...,US/Canada,Consumer,Web,Yes,Yes,80.00,5.00,1.00,5.00,1.00


In [9]:
for col in subscriber_info:
  print(col)

id
language
subscription_type
subscription_event_type
purchase_store
purchase_amount_raw
currency
subscription_start_date
subscription_expiration
demo_user
free_trial_user
free_trial_start_date
free_trial_expiration
auto_renew
country
user_type
lead_platform
email_subscriber
push_notifications
send_count
open_count
click_count
unique_open_count
unique_click_count


#### 2.2 typing columns
into cateogorical
into integer
into datetime

In [ ]:
subscriber_info["language"] = subscriber_info["language"].astype("category")
subscriber_info["subscription_type"] = subscriber_info["subscription_type"].astype("category")
subscriber_info["subscription_event_type"] = subscriber_info["subscription_event_type"].astype("category")
subscriber_info["purchase_store"] = subscriber_info["purchase_store"].astype("category")
subscriber_info["country"] = subscriber_info["country"].astype("category")
subscriber_info["user_type"] = subscriber_info["user_type"].astype("category")
subscriber_info["demo_user"] = subscriber_info["demo_user"].astype("category")
subscriber_info["currency"] = subscriber_info["currency"].astype("category")

In [ ]:
#convert true/false or yes/no to binary
def convert_yes_no(data, col):
  my_series = data[col].map({'Yes': 1, 'No': 0,
                             'On':1, 'Off':0})
  return my_series


subscriber_info["demo_user"] = convert_yes_no(subscriber_info,"demo_user").astype(bool)
subscriber_info["free_trial_user"] = convert_yes_no(subscriber_info,"free_trial_user").astype(bool)
subscriber_info["auto_renew"] = convert_yes_no(subscriber_info,"auto_renew").astype(bool)
subscriber_info["email_subscriber"] = convert_yes_no(subscriber_info,"email_subscriber").astype(bool)
subscriber_info["push_notifications"] = convert_yes_no(subscriber_info,"push_notifications").astype(bool)


#subscriber_info["demo_user"] = subscriber_info["demo_user"].astype(bool)
#subscriber_info["free_trial_user"] = subscriber_info["free_trial_user"].astype(bool)
#subscriber_info["auto_renew"] = subscriber_info["auto_renew"].astype(bool)
#subscriber_info["email_subscriber"] = subscriber_info["email_subscriber"].astype(bool)
#subscriber_info["push_notifications"] = subscriber_info["push_notifications"].astype(bool)

###### 2.2.1 date time typing

#### 2.3 app activity initial analysis

In [ ]:
app_activity["app_session_platform"].unique()

array(['ios', 'android', 'web', nan], dtype=object)

In [ ]:
app_activity["app_activity_type"].unique()

array(['App Launch', 'Start', 'Completed', 'Other', nan, 'Onboarding'],
      dtype=object)

In [ ]:
# strip all the leading and trailing white space in the Start Date and Expiration column 
subscriber_info['free_trial_start_date'] = subscriber_info['free_trial_start_date'].str.strip()
subscriber_info['free_trial_expiration'] = subscriber_info['free_trial_expiration'].str.strip()

# convert all date strings to datetime format 
subscriber_info['free_trial_start_date'] = pd.to_datetime(subscriber_info['free_trial_start_date'])
subscriber_info['free_trial_expiration'] = pd.to_datetime(subscriber_info['free_trial_expiration'])

# confirm conversion
print(subscriber_info.dtypes['free_trial_start_date'])
print(subscriber_info.dtypes['free_trial_expiration'])

datetime64[ns]
datetime64[ns]


In [ ]:
# strip all the leading and trailing white space in the Start Date and Expiration column 
subscriber_info['subscription_start_date'] = subscriber_info['subscription_start_date'].str.strip()
subscriber_info['subscription_expiration'] = subscriber_info['subscription_expiration'].str.strip()

# convert all date strings to datetime format 
subscriber_info['subscription_start_date'] = pd.to_datetime(subscriber_info['subscription_start_date'])
subscriber_info['subscription_expiration'] = pd.to_datetime(subscriber_info['subscription_expiration'])

# confirm conversion
print(subscriber_info.dtypes['subscription_start_date'])
print(subscriber_info.dtypes['subscription_expiration'])

datetime64[ns]
datetime64[ns]


In [ ]:
subscriber_info.columns

Index(['id', 'language', 'subscription_type', 'subscription_event_type',
       'purchase_store', 'purchase_amount_raw', 'currency',
       'subscription_start_date', 'subscription_expiration', 'demo_user',
       'free_trial_user', 'free_trial_start_date', 'free_trial_expiration',
       'auto_renew', 'country', 'user_type', 'lead_platform',
       'email_subscriber', 'push_notifications', 'send_count', 'open_count',
       'click_count', 'unique_open_count', 'unique_click_count'],
      dtype='object')

## 3.0 data transformations

simple data transformations that can have some beneficial information, such as subscription length

#### 3.1 converting foreign currency to usd

insert inline code to convert currency to use at date
https://pypi.org/project/CurrencyConverter/

In [ ]:
!pip install currencyconverter

     |████████████████████████████████| 544 kB 8.9 MB/s 


importing currency converter package, if rate is missing it goes to last known rate

In [ ]:
#from datetime import date # datetime works too
from currency_converter import CurrencyConverter
from datetime import date
c = CurrencyConverter(fallback_on_missing_rate=True)
#c.convert(100, 'EUR', 'USD', date=date(2013, 3, 21))

filling na values with USD, not for sure, but it makes sense for now. not too many missing

In [ ]:
subscriber_info["currency"].fillna("USD")
subscriber_info["currency"].replace(to_replace="nan",
                                                  value="USD")

def cur_filler(currency):
  if currency not in c.currencies:
    currency = 'USD'
  
  return currency

subscriber_info["currency"] = [cur_filler(cur) for cur in subscriber_info["currency"]]
#for val in subscriber_info.iloc[[index]]["currency"]

converting to USD using raw amount, foreign currency, and date of transaction

In [ ]:
subscriber_info["purchase_amount_usd"] = [c.convert(subscriber_info.iloc[[index]]["purchase_amount_raw"], 
                                                    subscriber_info.iloc[[index]]["currency"].values[0], 
                                                    'USD', 
                                                    date=subscriber_info.iloc[[index]]["subscription_start_date"].dt.date.values[0]) for index in range(0,subscriber_info.shape[0])]

#### 3.2 misc. transformations
creating subscription duration variable

In [ ]:
subscriber_info["subscription_length_days"] = (subscriber_info["subscription_expiration"] - subscriber_info["subscription_start_date"]).dt.days
subscriber_info["subscription_length_days"].head()

0    182
1     92
2    365
3     92
4     92
Name: subscription_length_days, dtype: int64

In [ ]:
# calculating minimum and maximum subscription to use as reference for later comparisons

# A large difference (> 2 or 3 years) between subscription_start_date and subscription_expiration indicates a valuable user 


difference  = (subscriber_info['subscription_expiration'] - subscriber_info['subscription_start_date'])
print(f'Minimum Difference: {difference.min()}')
print(f'Maximum Difference: {difference.max()}')
print(f'Average Difference: {difference.mean()}')

Minimum Difference: 30 days 00:00:00
Maximum Difference: 29312 days 00:00:00
Average Difference: 4918 days 11:03:00.798962624


In [ ]:
# Determing incorrect subscription periods. Subscription expiration cannot precede subscription start date!

# Remove all values that are True? 
(subscriber_info['subscription_expiration'] < subscriber_info['subscription_start_date']).sum()

0

## 4.0 limited vs lifetime membership eda
a customer is limited when their subscription has a predefined expiration date. this is not equivalent to an auto_renew subscription. a non-limited (perpetual subscription) is is someone who has a lifetime subscription and is denoted by their expiration date being in 2098 or 2099. this solution isn't good, and I understand that is exists likely due to SQL limitations.

subscription types, lifetime vs limited

In [ ]:
subscriber_info[["subscription_type","id"]].groupby("subscription_type").count()

,id
subscription_type,
Lifetime,6545
Limited,33557


In [ ]:
subscriber_info = pd.get_dummies(subscriber_info, columns=["subscription_type"])
subscriber_info.columns = subscriber_info.columns.str.lower()

In [ ]:
subscriber_info.columns

Index(['id', 'language', 'subscription_event_type', 'purchase_store',
       'purchase_amount_raw', 'currency', 'subscription_start_date',
       'subscription_expiration', 'demo_user', 'free_trial_user',
       'free_trial_start_date', 'free_trial_expiration', 'auto_renew',
       'country', 'user_type', 'lead_platform', 'email_subscriber',
       'push_notifications', 'send_count', 'open_count', 'click_count',
       'unique_open_count', 'unique_click_count', 'purchase_amount_usd',
       'subscription_length_days', 'subscription_type_lifetime',
       'subscription_type_limited'],
      dtype='object')

number of emails opened

if current demo user

In [ ]:
# proportion of emails opened

# variable if demo is expireed

## 5.0 merging dataframes

merging in app activity data

#### 5.1 merge app_activity to subscriber_info

merging number of times app started and total app interactions (including app starts). does this by ID

In [ ]:
app_launch_data = app_activity[app_activity['app_activity_type'] == "App Launch"]
launch_app_interactions = app_launch_data[["id","app_activity_type"]].groupby(['id']).count()
launch_app_interactions = launch_app_interactions.rename(columns={"app_activity_type":"launch_app_interactions"})

total_app_interactions = app_activity[['id',"app_activity_type"]].groupby(['id']).count()
total_app_interactions = total_app_interactions.rename(columns={"app_activity_type":"total_app_interactions"})

id_interactions = total_app_interactions.merge(launch_app_interactions, on="id")
id_interactions

,total_app_interactions,launch_app_interactions
id,,
1,14,12
2,80,27
3,78,39
4,38,15
5,152,63
...,...,...
39765,1,1
39845,1,1
39902,1,1


#### 5.2 mergins subscriber info and id interactions

In [ ]:
subscriber_info = subscriber_info.merge(id_interactions, on="id", how="left")
subscriber_info.head()

,id,language,subscription_event_type,purchase_store,purchase_amount_raw,currency,subscription_start_date,subscription_expiration,demo_user,free_trial_user,...,open_count,click_count,unique_open_count,unique_click_count,purchase_amount_usd,subscription_length_days,subscription_type_lifetime,subscription_type_limited,total_app_interactions,launch_app_interactions
0,1,POR,INITIAL_PURCHASE,App,NaN,USD,2018-12-28,2019-06-28,True,False,...,7.00,0.00,6.00,0.00,NaN,182,0,1,14.00,12.00
1,2,EBR,INITIAL_PURCHASE,Web,39.00,USD,2019-11-28,2020-02-28,False,False,...,3.00,0.00,1.00,0.00,39.00,92,0,1,80.00,27.00
2,3,ESP,INITIAL_PURCHASE,Web,0.00,USD,2018-12-31,2019-12-31,False,False,...,0.00,0.00,0.00,0.00,0.00,365,0,1,78.00,39.00
3,4,KOR,INITIAL_PURCHASE,App,NaN,USD,2019-11-07,2020-02-07,True,False,...,0.00,0.00,0.00,0.00,NaN,92,0,1,38.00,15.00
4,5,ENG,INITIAL_PURCHASE,App,NaN,USD,2019-08-13,2019-11-13,False,False,...,5.00,1.00,5.00,1.00,NaN,92,0,1,152.00,63.00


In [ ]:
print("# nulls for app interacitons:",subscriber_info["total_app_interactions"].isna().sum())
print("# nulls for launch interacitons:",subscriber_info["launch_app_interactions"].isna().sum())

# nulls for app interacitons: 17189
# nulls for launch interacitons: 17189


#### 5.3 dropping un needed columns
some columns not totally neccesary for the analysis, maybe using them in the future may be useful

In [ ]:
subscriber_info.drop(columns=["purchase_amount_raw","currency","free_trial_start_date","free_trial_expiration","open_count","click_count"], 
                            inplace=True)

## 6.0 purchase_amount analysis and padding
the standard deviation is 11,606,125.17, not very likely

the mean is 2,106,351.47

and the max is 135,792,683.00, no chance. the market cap 737.78 million. no chance that they made a 135 million. 

to combat this, I'll train a standard z-score scaler to values under 1000, then I'll look 


In [ ]:
from sklearn.linear_model import LinearRegression
from sklearn.preprocessing import StandardScaler

In [ ]:
subscriber_info["purchase_amount_usd"].describe()

count           26923.00
mean        211607745.60
std       15701993095.50
min                 0.00
25%                 0.00
50%                39.00
75%               160.80
max     1600000000000.00
Name: purchase_amount_usd, dtype: float64

removing extreme observations, now the most expensive purchase was 1000 dollars. if there really were purchase amounts over this, it is unlikely that a linear model (or something similar) will be able to accurately portray those numbers. but, overall, it seems unlikely that Rosetta Stone gathered a significant amount of its revenue from purchases over $1000. 

I would use a log-norm distribution because it stems from zero, but I don't know much about it and how to implement it. a gaussian (normal) distribution will suffice

In [ ]:
z = StandardScaler()

# fitting the scaler on values that are less than $5000, this allows us to build a z score that will easily be able to weed out high values
# this assumes that few, if any people spend more than 5 grand on Rosetta Stone
z.fit(subscriber_info.loc[((subscriber_info["purchase_amount_usd"] <= 5000) & (subscriber_info["purchase_amount_usd"] >= 0.01)),
                          "purchase_amount_usd"].values.reshape(-1,1))

#df.loc[((df['col1'] == 'A') & (df['col2'] == 'G'))]

StandardScaler()

In [ ]:
# 4 standard distributions represents a tiny amount of the data
z_thresh = 4

subscriber_info["purchase_amount_usd_z"] = z.transform(subscriber_info["purchase_amount_usd"].values.reshape(-1,1))

inaccurate_prices = subscriber_info["purchase_amount_usd_z"] > z_thresh

subscriber_info.loc[inaccurate_prices,["purchase_amount_usd_z","purchase_amount_usd"]] = np.nan

print("num values removed:", (subscriber_info["id"].count() - subscriber_info["purchase_amount_usd_z"].count()))

num values removed: 14678


confirming that values over crazy purchase prices are nan. also, the highest purcahse price is 400 dollars. the previous transformation removed the highest values.

In [ ]:
subscriber_info[subscriber_info["purchase_amount_usd_z"] > z_thresh]["purchase_amount_usd_z"].count()

subscriber_info.drop("purchase_amount_usd_z", axis=1, inplace=True)

In [ ]:
subscriber_info["purchase_amount_usd"].describe()

count   25424.00
mean       70.50
std        77.84
min         0.00
25%         0.00
50%        36.85
75%       119.00
max       412.97
Name: purchase_amount_usd, dtype: float64

#### 6.1 modelling for the incorrect data

using variables to predict what the real price might be, the data had differing numbers of trailing zeros and the return for doing a fix is minimal. 

In [ ]:
subscriber_info.isna().sum(axis=0)

id                                0
language                          0
subscription_event_type           0
purchase_store                    0
subscription_start_date           0
subscription_expiration           0
demo_user                         0
free_trial_user                   0
auto_renew                        0
country                           0
user_type                         0
lead_platform                     0
email_subscriber                  0
push_notifications                0
send_count                    11654
unique_open_count             11654
unique_click_count            11654
purchase_amount_usd           14678
subscription_length_days          0
subscription_type_lifetime        0
subscription_type_limited         0
total_app_interactions        17189
launch_app_interactions       17189
dtype: int64

In [ ]:
print("num nan prices:",np.count_nonzero(np.isnan(subscriber_info["purchase_amount_usd"])))

num nan prices: 14678


In [ ]:
subscriber_info.isnull().sum(axis=0)

id                                0
language                          0
subscription_event_type           0
purchase_store                    0
subscription_start_date           0
subscription_expiration           0
demo_user                         0
free_trial_user                   0
auto_renew                        0
country                           0
user_type                         0
lead_platform                     0
email_subscriber                  0
push_notifications                0
send_count                    11654
unique_open_count             11654
unique_click_count            11654
purchase_amount_usd           14678
subscription_length_days          0
subscription_type_lifetime        0
subscription_type_limited         0
total_app_interactions        17189
launch_app_interactions       17189
dtype: int64

In [ ]:
# fill na of interaction counts with 0
# if not known assume they are zero
values = {"send_count": 0, "open_count": 0, "click_count": 0, "unique_open_count": 0, "unique_click_count": 0, "total_app_interactions": 0,"launch_app_interactions": 0}
subscriber_info = subscriber_info.fillna(value=values)


In [ ]:
price_replacement_lm = LinearRegression()

nan_prices = subscriber_info['purchase_amount_usd'].isna()

price_train_data = subscriber_info.loc[~nan_prices]
price_fill_data = subscriber_info.loc[nan_prices]

predictors = ['language', 'subscription_event_type','purchase_store', 'demo_user',
              'free_trial_user','auto_renew', 'country', 'user_type', 'email_subscriber', 
              'push_notifications', 'unique_open_count', 'unique_click_count', 
              'subscription_length_days', 'total_app_interactions', 'launch_app_interactions',
              'subscription_type_lifetime', 'subscription_type_limited']

outcome = ["purchase_amount_usd"]

price_replacement_lm.fit(pd.get_dummies(price_train_data[predictors]), price_train_data[outcome])
train_preds = price_replacement_lm.predict(pd.get_dummies(price_train_data[predictors]))
test_preds = price_replacement_lm.predict(pd.get_dummies(price_fill_data[predictors]))

In [ ]:
#lm_coefficients = pd.DataFrame({"Coef":price_replacement_lm.coef_,
#                             "Name": price_train_data[predictors].columns})
#lm_coefficients = lm_coefficients.append({"Coef": price_replacement_lm.intercept_,
#                                   "Name": "intercept"}, ignore_index = True)

In [ ]:
print("train mse: ",mean_squared_error(price_train_data[outcome], train_preds))
print("train r2 score:", r2_score(price_train_data[outcome], train_preds))

train mse:  1467.1067773240468
train r2 score: 0.7578342435956755


In [ ]:
subscriber_info["purchase_amount_usd_imputed"] = -1

In [ ]:
subscriber_info.loc[~nan_prices,"purchase_amount_usd_imputed"] = subscriber_info.loc[~nan_prices]["purchase_amount_usd"]
subscriber_info.loc[nan_prices,"purchase_amount_usd_imputed"] = np.array(test_preds)

subscriber_info["purchase_amount_usd_imputed"] = np.maximum(subscriber_info["purchase_amount_usd_imputed"], 0)

## 7.0 exporting data

In [ ]:
#selecting a few important columns
subscriber_info = subscriber_info.drop(columns=["id","subscription_expiration"])

subscriber_info.to_csv("/content/drive/MyDrive/410 Group Project/data/rosetta_stone_clean.csv")

In [ ]:
from google.colab import files
files.download("/content/drive/MyDrive/410 Group Project/data/rosetta_stone_clean.csv")

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>